In [105]:
import pandas as pd

In [106]:
old_df = pd.read_csv("model_props_with_r2_v1.csv")

new_r2_vals = pd.read_csv("alignment_results_full.csv")
new_props = pd.read_csv("../data/model_properties/model_attributes_updated_v8.csv")

In [107]:
new_props

,name,family,parameters,activation_function,instruct_tuned,architecture,modality,fine_tuned,training_tokens,n_layers,...,vocab_size,max_pos_embed,intermediate_size,average,ifeval,bbh,math,gpqa,musr,mmlu
0,gemma-2-2b,gemma,2B,GeGLU,False,decoder,text,False,2T,26,...,256000,8192,9216,10.13,19.93,11.76,2.87,1.68,11.43,13.11
1,gemma-2-2b-it,gemma,2B,GeGLU,True,decoder,text,True,2T,26,...,256000,8192,9216,17.05,56.68,17.98,0.08,3.24,7.08,17.22
2,gemma-2-9b,gemma,9B,GeGLU,False,decoder,text,False,8T,42,...,25600,8192,14336,21.21,20.40,34.10,13.44,10.51,14.30,34.48
3,gemma-2-9b-it,gemma,9B,GeGLU,True,decoder,text,True,8T,42,...,25600,8192,14336,32.07,74.36,42.14,19.49,14.77,9.74,31.95
4,gemma-2-27b,gemma,27B,GeGLU,False,decoder,text,False,13T,46,...,256000,8192,36864,23.93,24.75,37.39,16.62,13.42,13.92,37.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Phi-3.5-MoE-instruct,phi,42B,SwiGLU,True,decoder,text,True,4.9T,32,...,32064,131072,6400,36.88,69.25,48.77,31.19,14.09,17.33,40.64
72,Phi-3.5-vision-instruct,phi,4.1B,SwiGLU,True,decoder,multimodal,True,0.5T,32,...,32064,131072,8192,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Qwen3-4B-Base,qwen,4B,SwiGLU,False,decoder,text,False,36T,36,...,151936,32768,9728,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,qwen3-8b,qwen,8B,SwiGLU,True,decoder,text,True,36T,36,...,151936,40960,12288,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
new_r2_vals

,model,r2,rdm_corr,cka
0,model_triplet_output_qwen25-14b-instruct_THING...,0.491798,0.636479,0.702113
1,model_triplet_output_Phi-3-medium-4k-instruct_...,0.468913,0.647438,0.642572
2,model_triplet_output_Phi-3.5-MoE-instruct_THIN...,0.467597,0.601368,0.649253
3,model_triplet_output_gemma-3-27b-it_THINGS_emb...,0.459731,0.613899,0.642256
4,model_triplet_output_qwen25-7b-instruct_THINGS...,0.459009,0.594460,0.665128
...,...,...,...,...
77,model_triplet_output_gpt-oss-20b_THINGS_embedding,0.086543,-0.010191,0.084147
78,model_triplet_output_gemma-3-270m-it_THINGS_em...,0.085273,0.012516,0.074579
79,model_triplet_output_mamba-1.4b-hf_THINGS_embe...,0.082597,0.001555,0.079291
80,model_triplet_output_mamba-2.8b-hf_THINGS_embe...,0.076694,-0.013858,0.066422


In [109]:
# Remove prefix and suffix from model column
new_r2_vals["model_clean"] = (
    new_r2_vals["model"]
    .str.replace(r"^model_triplet_output_", "", regex=True)
    .str.replace(r"_THINGS.*embedding$", "", regex=True)
)

print(new_r2_vals[["model", "model_clean"]].head())


                                               model               model_clean
0  model_triplet_output_qwen25-14b-instruct_THING...       qwen25-14b-instruct
1  model_triplet_output_Phi-3-medium-4k-instruct_...  Phi-3-medium-4k-instruct
2  model_triplet_output_Phi-3.5-MoE-instruct_THIN...      Phi-3.5-MoE-instruct
3  model_triplet_output_gemma-3-27b-it_THINGS_emb...            gemma-3-27b-it
4  model_triplet_output_qwen25-7b-instruct_THINGS...        qwen25-7b-instruct


In [110]:
#  Merge with suffixes to distinguish overlaps
merged_df = pd.merge(old_df, new_props, on="name", how="left", suffixes=("", "_new"))

# Drop the duplicate columns from new_props
dup_cols = [c for c in merged_df.columns if c.endswith("_new")]
merged_df = merged_df.drop(columns=dup_cols)

In [111]:
len(merged_df)

65

In [112]:
# Make temporary join keys: first 4 digits of r2
merged_df["r2_key"] = merged_df["r2"].round(4)
new_r2_vals["r2_key"] = new_r2_vals["r2"].round(4)

# Merge on r2_key
merged_df = merged_df.merge(
    new_r2_vals[["r2_key", "rdm_corr", "cka"]],
    on="r2_key",
    how="left"
)

# Drop the temp key if you don’t want it
merged_df = merged_df.drop(columns=["r2_key"])

print(merged_df.head())

# Export
merged_df.to_csv("merged_with_rdm_cka.csv", index=False)


            name family parameters activation_function  instruct_tuned  \
0     gemma-2-2b  gemma         2B               GeGLU           False   
1  gemma-2-2b-it  gemma         2B               GeGLU            True   
2     gemma-2-9b  gemma         9B               GeGLU           False   
3  gemma-2-9b-it  gemma         9B               GeGLU            True   
4    gemma-2-27b  gemma        27B               GeGLU           False   

  architecture modality  fine_tuned  average  ifeval  ...  n_layers  n_heads  \
0      decoder     text       False    10.13   19.93  ...        26        8   
1      decoder     text        True    17.05   56.68  ...        26        8   
2      decoder     text       False    21.21   20.40  ...        42       16   
3      decoder     text        True    32.07   74.36  ...        42       16   
4      decoder     text       False    23.93   24.75  ...        46       32   

   n_kv_heads  attn_dim  hidden_size  vocab_size max_pos_embed  \
0       

In [113]:
merged_df.head()

,name,family,parameters,activation_function,instruct_tuned,architecture,modality,fine_tuned,average,ifeval,...,n_layers,n_heads,n_kv_heads,attn_dim,hidden_size,vocab_size,max_pos_embed,intermediate_size,rdm_corr,cka
0,gemma-2-2b,gemma,2B,GeGLU,False,decoder,text,False,10.13,19.93,...,26,8,4,256,2304,256000,8192,9216,0.034266,0.140113
1,gemma-2-2b-it,gemma,2B,GeGLU,True,decoder,text,True,17.05,56.68,...,26,8,4,256,2304,256000,8192,9216,0.429187,0.483059
2,gemma-2-9b,gemma,9B,GeGLU,False,decoder,text,False,21.21,20.40,...,42,16,8,256,3584,25600,8192,14336,0.185771,0.237628
3,gemma-2-9b-it,gemma,9B,GeGLU,True,decoder,text,True,32.07,74.36,...,42,16,8,256,3584,25600,8192,14336,0.579114,0.664769
4,gemma-2-27b,gemma,27B,GeGLU,False,decoder,text,False,23.93,24.75,...,46,32,16,128,4608,256000,8192,36864,0.527818,0.585488


In [114]:
merged = new_props.merge(
    new_r2_vals,
    left_on="name",
    right_on="model_clean",
    how="left"
)

print(merged.head())
print(len(merged))

            name family parameters activation_function  instruct_tuned  \
0     gemma-2-2b  gemma         2B               GeGLU           False   
1  gemma-2-2b-it  gemma         2B               GeGLU            True   
2     gemma-2-9b  gemma         9B               GeGLU           False   
3  gemma-2-9b-it  gemma         9B               GeGLU            True   
4    gemma-2-27b  gemma        27B               GeGLU           False   

  architecture modality  fine_tuned training_tokens  n_layers  ...   math  \
0      decoder     text       False              2T        26  ...   2.87   
1      decoder     text        True              2T        26  ...   0.08   
2      decoder     text       False              8T        42  ...  13.44   
3      decoder     text        True              8T        42  ...  19.49   
4      decoder     text       False             13T        46  ...  16.62   

    gpqa   musr   mmlu                                              model  \
0   1.68  11.43

In [115]:
fill_gap = pd.read_csv("model_props_with_r2_v5.csv")

In [116]:
fill_gap

,name,family,parameters,activation_function,instruct_tuned,architecture,modality,fine_tuned,average,ifeval,...,n_layers,n_heads,n_kv_heads,attn_dim,hidden_size,vocab_size,max_pos_embed,intermediate_size,rdm_corr,cka
0,gemma-2-2b,gemma,2B,GeGLU,False,decoder,text,False,10.13,19.93,...,26,8,4,256,2304,256000,8192,9216,0.034266,0.140113
1,gemma-2-2b-it,gemma,2B,GeGLU,True,decoder,text,True,17.05,56.68,...,26,8,4,256,2304,256000,8192,9216,0.429187,0.483059
2,gemma-2-9b,gemma,9B,GeGLU,False,decoder,text,False,21.21,20.40,...,42,16,8,256,3584,25600,8192,14336,0.185771,0.237628
3,gemma-2-9b-it,gemma,9B,GeGLU,True,decoder,text,True,32.07,74.36,...,42,16,8,256,3584,25600,8192,14336,0.579114,0.664769
4,gemma-2-27b,gemma,27B,GeGLU,False,decoder,text,False,23.93,24.75,...,46,32,16,128,4608,256000,8192,36864,0.527818,0.585488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,amd/Instella-3B,Instella,3B,NaN,False,decoder,text,False,NaN,NaN,...,36,32,32,80,2560,50304,4096,6912,0.367493,0.417895
69,amd/Instella-3B-SFT,Instella,3B,NaN,True,decoder,text,True,NaN,NaN,...,36,32,32,80,2560,50304,4096,6912,0.449639,0.427776
70,AMD-OLMo-1B-SFT-DPO,OLMo,1B,SwiGLU,True,decoder,text,True,NaN,NaN,...,16,16,16,128,2048,50304,2048,8192,0.071649,0.115362
71,AMD-OLMo-1B,OLMo,1B,SwiGLU,False,decoder,text,False,NaN,NaN,...,16,16,16,128,2048,50304,2048,8192,0.051264,0.130871


In [117]:
merged

,name,family,parameters,activation_function,instruct_tuned,architecture,modality,fine_tuned,training_tokens,n_layers,...,math,gpqa,musr,mmlu,model,r2,rdm_corr,cka,model_clean,r2_key
0,gemma-2-2b,gemma,2B,GeGLU,False,decoder,text,False,2T,26,...,2.87,1.68,11.43,13.11,model_triplet_output_gemma-2-2b_THINGS_embedding,0.125507,0.034266,0.140113,gemma-2-2b,0.1255
1,gemma-2-2b-it,gemma,2B,GeGLU,True,decoder,text,True,2T,26,...,0.08,3.24,7.08,17.22,model_triplet_output_gemma-2-2b-it_THINGS_embe...,0.320657,0.429187,0.483059,gemma-2-2b-it,0.3207
2,gemma-2-9b,gemma,9B,GeGLU,False,decoder,text,False,8T,42,...,13.44,10.51,14.30,34.48,model_triplet_output_gemma-2-9b_THINGS_embedding,0.177993,0.185771,0.237628,gemma-2-9b,0.1780
3,gemma-2-9b-it,gemma,9B,GeGLU,True,decoder,text,True,8T,42,...,19.49,14.77,9.74,31.95,model_triplet_output_gemma-2-9b-it_THINGS_embe...,0.439693,0.579114,0.664769,gemma-2-9b-it,0.4397
4,gemma-2-27b,gemma,27B,GeGLU,False,decoder,text,False,13T,46,...,16.62,13.42,13.92,37.45,model_triplet_output_gemma-2-27b_THINGS_embedding,0.368882,0.527818,0.585488,gemma-2-27b,0.3689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Phi-3.5-MoE-instruct,phi,42B,SwiGLU,True,decoder,text,True,4.9T,32,...,31.19,14.09,17.33,40.64,model_triplet_output_Phi-3.5-MoE-instruct_THIN...,0.467597,0.601368,0.649253,Phi-3.5-MoE-instruct,0.4676
73,Phi-3.5-vision-instruct,phi,4.1B,SwiGLU,True,decoder,multimodal,True,0.5T,32,...,NaN,NaN,NaN,NaN,model_triplet_output_Phi-3.5-vision-instruct_T...,0.399620,0.459660,0.565391,Phi-3.5-vision-instruct,0.3996
74,Qwen3-4B-Base,qwen,4B,SwiGLU,False,decoder,text,False,36T,36,...,NaN,NaN,NaN,NaN,model_triplet_output_Qwen3-4B-Base_THINGS_embe...,0.193860,0.227185,0.273683,Qwen3-4B-Base,0.1939
75,qwen3-8b,qwen,8B,SwiGLU,True,decoder,text,True,36T,36,...,NaN,NaN,NaN,NaN,model_triplet_output_qwen3-8b_THINGS_embedding,0.388093,0.521510,0.565576,qwen3-8b,0.3881


In [119]:
print(merged["r2"].isna().sum())


30


In [120]:
# Subset fill_gap to only the columns we need
fg_sub = fill_gap[["name", "r2", "rdm_corr", "cka"]]

# Merge only missing r2 rows with fill_gap
mask = merged["r2"].isna()
to_fill = merged.loc[mask, ["name"]].merge(fg_sub, on="name", how="left")

# Update values back into merged
for col in ["r2", "rdm_corr", "cka"]:
    merged.loc[mask, col] = to_fill[col].values


In [ ]:
print(merged["r2"].isna().sum())


4


In [123]:
# Show rows where r2 is still NaN
print(merged[merged["r2"].isna()])


                           name family parameters activation_function  \
36      allenai/OLMo-2-1124-13B   OLMo        13B              SwiGLU   
37  allenai/OLMo-2-1124-13B-SFT   OLMo        13B              SwiGLU   
38  allenai/OLMo-2-1124-13B-DPO   OLMo        13B              SwiGLU   
54   state-spaces/mamba-2.8b-hf  mamba         3B                 NaN   

    instruct_tuned architecture modality  fine_tuned training_tokens  \
36           False      decoder     text       False              5T   
37            True      decoder     text        True              5T   
38            True      decoder     text        True              5T   
54           False          ssm     text       False            0.3T   

    n_layers  ...  math  gpqa  musr  mmlu  model  r2  rdm_corr  cka  \
36        40  ...   NaN   NaN   NaN   NaN    NaN NaN       NaN  NaN   
37        40  ...   NaN   NaN   NaN   NaN    NaN NaN       NaN  NaN   
38        40  ...   NaN   NaN   NaN   NaN    NaN NaN       N

In [124]:
merged.to_csv("model_props_with_r2_final.csv", index=False)